## 特征变换概述

### 为什么需要特征缩放与转换？

在真实数据集中，不同特征可能具有不同的量纲与取值范围，例如：
- `age` 通常在 20–100 范围内；
- `balance` 可从负数到上万；
- `duration`（通话时长）单位是秒；
- 而 `job`、`education` 等为类别型特征。

许多机器学习算法（如 KNN、SVM、Logistic Regression、梯度下降优化）都对输入特征的**尺度敏感**，如果不做特征缩放与转换，会出现以下问题：
- 欧氏距离计算失真（KNN、K-Means）；
- 梯度更新不稳定（Logistic Regression）；
- 正则化不公平（惩罚大数值特征更严重）；
- 模型收敛缓慢、性能下降。

### 本模块将介绍以下 4 种常见转换方式：
1. **标准化（Standardization）**：转换为均值为 0、方差为 1 的正态分布；
2. **最小–最大缩放（Min-Max Scaling）**：线性映射到指定区间（通常是 [0, 1]）；
3. **One-Hot 编码（One-Hot Encoding）**：将类别变量转换为 0/1 二值列；
4. **分箱（Binning）**：将连续变量按区间分段转换为离散类别。

每种转换方式将配有适当列的示例与代码实现。



本notebook使用bank.csv来进行演示

In [7]:
import pandas as pd

df = pd.read_csv('bank.csv',sep=';')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


## 标准化（Standardization）

### 🎯 标准化的目的
- 将数值特征转换为均值为 0、标准差为 1 的分布，即：
  $$
  z = \frac{x - \mu}{\sigma}
  $$
- 标准化后特征处于统一尺度，适用于对距离敏感的模型（如 KNN、SVM）或依赖梯度优化的模型（如逻辑回归、神经网络）。

### 🧠 注意事项
- 应只对**连续型数值特征**使用标准化（如 `age`, `balance`, `duration`）。  
- 对于有大量异常值的列，标准化可能不如稳健缩放（如中位数缩放）效果好。  
- 应该在训练集上 fit（学习均值和方差），再在训练/测试集上 transform，避免数据泄漏。


In [8]:
from sklearn.preprocessing import StandardScaler

# 选择需要标准化的列
cols_to_standardize = ['age', 'duration']

scaler = StandardScaler()
scaled_array = scaler.fit_transform(df[cols_to_standardize])

# 转换为 DataFrame
df_standardized = pd.DataFrame(scaled_array, columns=[f"{col}_z" for col in cols_to_standardize])

# 拼接原始列与标准化结果作对比
comparison = pd.concat([df[cols_to_standardize], df_standardized], axis=1)
comparison.describe()

,age,duration,age_z,duration_z
count,4521.000000,4521.000000,4.521000e+03,4.521000e+03
mean,41.170095,263.961292,-1.178737e-16,-7.622500e-17
std,10.576211,259.856633,1.000111e+00,1.000111e+00
min,19.000000,4.000000,-2.096455e+00,-1.000513e+00
25%,33.000000,104.000000,-7.725828e-01,-6.156433e-01
50%,39.000000,185.000000,-2.052091e-01,-3.038984e-01
75%,49.000000,329.000000,7.404137e-01,2.503146e-01
max,87.000000,3025.000000,4.333780e+00,1.062641e+01


## 最小–最大缩放（Min–Max Scaling）

### ✅ 什么是最小–最大缩放？
该方法将数值特征线性压缩到指定区间（通常是 [0, 1]）：

$$
x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}
$$

它保持原始数据的分布形状与相对间距，但统一了量纲，适合以下场景：

- 神经网络模型（如 MLP、CNN）；
- 对输入特征范围有限制的模型（如树模型中分箱可视化）；
- 特征分布不含异常值或区间清晰界定（如百分比、时间长度等）。

### 📌 特点
- 易受极值影响；
- 可设定输出区间（如 [0, 1]、[-1, 1]）；
- 不改变数据的分布形状。

### 🛠 推荐适用列（来自 bank.csv）
- `balance`：账户余额，范围差距较大；
- `duration`：也可做 Min-Max 尝试用于模型对比；


In [9]:
from sklearn.preprocessing import MinMaxScaler

# 使用 bank.csv 的 balance 和 duration 进行缩放
cols_to_scale = ['balance', 'duration']
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(df[cols_to_scale])

# 构造缩放后 DataFrame
df_minmax = pd.DataFrame(scaled_array, columns=[f"{col}_scaled" for col in cols_to_scale], index=df.index)

# 拼接原始与缩放后结果
comparison_minmax = pd.concat([df[cols_to_scale], df_minmax], axis=1)
print(comparison_minmax.head())


   balance  duration  balance_scaled  duration_scaled
0     1787        79        0.068455         0.024826
1     4789       220        0.108750         0.071500
2     1350       185        0.062590         0.059914
3     1476       199        0.064281         0.064548
4        0       226        0.044469         0.073486


## One-Hot 编码（One-Hot Encoding）

### ✅ 什么是 One-Hot 编码？
One-Hot 编码是将类别特征转换为二进制向量的一种方式。  
对每个可能的类别值，创建一个新的二元列，标记该样本是否属于该类别：

例如：

education = "secondary" → [0, 1, 0]

education = "tertiary" → [0, 0, 1]


### 📌 作用与意义
- 将非数值型类别特征转换为可被模型识别的格式；
- 避免类别型变量被错误地解释为有序数值；
- 特别适用于线性模型、树模型、深度学习模型等。

### 🧠 注意事项
- 类别数较多时会导致维度膨胀（High Dimensionality）；
- 可使用 `drop='first'` 避免多重共线性（线性模型常用）；
- 对于频繁变更的数据，可考虑使用 Target Encoding、Frequency Encoding 等方法替代。

### 🛠 推荐适用列（来自 bank.csv）
- `job`, `marital`, `education`, `default`, `housing`, `loan`, `contact`, `month`, `poutcome` 等


In [12]:
from sklearn.preprocessing import OneHotEncoder

# 选择几列类别变量示例
categorical_cols = ['job', 'marital', 'education']

# 使用 sparse_output=False 替代旧参数 sparse=False
encoder = OneHotEncoder(sparse_output=False, drop=None)
encoded_array = encoder.fit_transform(df[categorical_cols])

# 构建编码后的 DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(categorical_cols), index=df.index)

# 拼接原始与编码列
comparison_ohe = pd.concat([df[categorical_cols], encoded_df], axis=1)
print(comparison_ohe.head())


           job  marital  education  job_admin.  job_blue-collar  \
0   unemployed  married    primary         0.0              0.0   
1     services  married  secondary         0.0              0.0   
2   management   single   tertiary         0.0              0.0   
3   management  married   tertiary         0.0              0.0   
4  blue-collar  married  secondary         0.0              1.0   

   job_entrepreneur  job_housemaid  job_management  job_retired  \
0               0.0            0.0             0.0          0.0   
1               0.0            0.0             0.0          0.0   
2               0.0            0.0             1.0          0.0   
3               0.0            0.0             1.0          0.0   
4               0.0            0.0             0.0          0.0   

   job_self-employed  ...  job_technician  job_unemployed  job_unknown  \
0                0.0  ...             0.0             1.0          0.0   
1                0.0  ...             0.0     

## 分箱（Binning）

### ✅ 什么是分箱？
分箱（Binning）是将连续数值型特征划分为若干个离散的区间（bin），从而将其转换为类别变量。例如：

- 将 `age` 按照年龄段分成：
  - `<=25`：青年
  - `26–60`：中年
  - `>60`：老年

这种方式对以下任务特别有用：
- 将数值特征用于决策树模型；
- 对某些分布较偏或有明显分段意义的变量做转换；
- 提升模型解释性（例如规则提取、风险等级划分）。

### 🔧 分箱方式
1. **等宽分箱（uniform width）**：将取值范围均匀划分为 n 段；
2. **等频分箱（quantile binning）**：确保每个分箱内样本数量大致相等；
3. **自定义边界分箱**：根据业务逻辑自定义每个 bin 的边界；
4. **使用 `pd.cut`（等宽） 或 `pd.qcut`（等频）** 实现。

### 🛠 推荐适用列（来自 bank.csv）
- `age`：适合划分年龄段；
- `balance`：划分资产等级；


In [13]:
# 对 age 进行等宽和等频分箱

# 等宽分箱：将 age 分成 3 个区间
df['age_bin_uniform'] = pd.cut(df['age'], bins=3, labels=['young', 'middle', 'old'])

# 等频分箱：将 age 按分位数分成 4 个区间
df['age_bin_quantile'] = pd.qcut(df['age'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# 展示原始 age 和两种分箱结果
print(df[['age', 'age_bin_uniform', 'age_bin_quantile']].head(10))


   age age_bin_uniform age_bin_quantile
0   30           young               Q1
1   33           young               Q1
2   35           young               Q2
3   30           young               Q1
4   59          middle               Q4
5   35           young               Q2
6   36           young               Q2
7   39           young               Q2
8   41           young               Q3
9   43          middle               Q3


## Block 7：特征缩放与转换总结

在机器学习前的数据预处理中，特征缩放与转换是一项关键任务。它不仅影响模型的性能、训练稳定性，还决定了模型对不同特征的感知能力。

---

### 📌 方法总览与适用性对比

| 方法                | 转换对象        | 是否保持原分布 | 是否受异常值影响 | 适用模型                     | 特点与建议                         |
|---------------------|-----------------|----------------|------------------|------------------------------|------------------------------------|
| 标准化（Z-score）    | 连续数值特征     | ❌（转为标准正态） | ✅                | KNN, SVM, Logistic, NN       | 对离群值敏感；需在训练集上 fit     |
| Min-Max 缩放         | 连续数值特征     | ✅              | ✅                | NN、图像数据、正则系数敏感模型  | 易受极端值影响；建议先除离群值     |
| One-Hot 编码         | 类别特征         | ✅              | ❌                | 所有模型                     | 类别数大时维度膨胀，建议配合 PCA 等 |
| 分箱（Binning）      | 连续数值特征 → 离散 | ❌              | ❌                | 决策树、规则提取模型           | 可提升可解释性，但信息损失较大     |

---

### ✅ 实战建议

1. **选择变换方式需考虑模型类型**  
   - 线性模型、神经网络 → 标准化 / Min-Max；
   - 树模型（如 XGBoost、Random Forest） → 通常不需要缩放；
   - 类别型特征 → 使用 One-Hot 编码或 Target Encoding（高基数时）；

2. **流水线一致性与避免数据泄漏**
   - 所有 `fit` 操作（如 `StandardScaler().fit()`）必须在训练集上完成；
   - 测试集与新数据只使用 `.transform()`；
   - 推荐使用 `Pipeline` 保证一致处理流程。

3. **分箱需谨慎**
   - 若模型可直接处理连续变量（如逻辑回归、树模型），不建议盲目分箱；
   - 若需解释性（如评分卡模型、风控模型），可配合业务规则进行分箱。

---

### 🧠 最佳实践组合（举例）

- 对连续数值特征：
  - `StandardScaler()` + PCA（用于降维）；
  - `MinMaxScaler()` + 神经网络；
- 对类别变量：
  - `OneHotEncoder()` 用于类别不多时；
  - `pd.cut()` + Label Encoding 用于分箱后继续建模；
- 使用 `ColumnTransformer` 同时处理数值列和类别列。

